## Homework 4

### Model optimization using GridSearchCV

1. Using sklearn.datasets.load_boston, make a pipeline that combines
    - StandardScaler
    - KernelPCA with "poly" kernel
    - ElasticNet regression
2. Find the best combination of the following 4 parameters:
    - KernelPCA.gamma 
    - KernelPCA.degree
    - ElasticNet.alpha
    - ElasticNet.l1_ratio
   Make nice plots to support your optimization procedure.
3. Learn from Scikit-Learn documentation how to use different models as parameter values. 
   Make a pipeline with StandardScaler, KernelPCA and a "regressor", then compare
   classifiers like "Ridge", "RandomForestRegressor" and "sklearn.smv.SVR". 
   Which classifier is the best?

In [ ]:
# Load boston and get X(data) and y(target)
from sklearn.datasets import load_boston
X, y = load_boston(return_X_y=True)

from sklearn.model_selection import train_test_split
X, X_test, y, y_test = train_test_split(X, y, test_size=0.2, random_state=30)
display(X.shape, y_test.shape)


In [ ]:
# Make a pipeline that includes StandardScaler, KernelPCA with "poly" kernel and ElasticNet regression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import KernelPCA
from sklearn.linear_model import ElasticNet

pipe = Pipeline(
    steps = [
        ('SScaler', StandardScaler()),
        ('KPCA', KernelPCA(n_components=3, kernel='poly')),
        ('ENet', ElasticNet())
    ]
)

pipe.fit(X, y)
pipe.score(X,y)

In [ ]:
pipe.score(X_test,y_test)

In [ ]:
from sklearn.model_selection import GridSearchCV
grid_params = [{
    'KPCA__gamma': [0.0001, 0.001, 0.01, 0.1, 1],
    'KPCA__degree': [1, 3, 6, 9, 12],
    'ENet__alpha': [0.00001, 0.0001, 0.01, 0.1, 1, 10],
    'ENet__l1_ratio': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
    
}]
GCV = GridSearchCV(pipe, grid_params, cv=5, n_jobs=-1, verbose=10)

In [ ]:
GCV.fit(X, y)

In [ ]:
GCV.best_params_

In [ ]:
GCV.score(X,y)

In [ ]:
GCV.score(X_test,y_test)

In [ ]:
import pandas as pd
import numpy as np
params = ["KPCA__gamma", "KPCA__degree", "ENet__alpha", "ENet__l1_ratio"]
p = params[0]
q = params[1]
y_axis = GCV.param_grid[0][p]
y_index = GCV.cv_results_["param_" + p]

x_axis = GCV.param_grid[0][q]
x_index = GCV.cv_results_["param_" + q]


heatmap = pd.DataFrame(index=y_axis, columns=x_axis, dtype=np.float)
heatmap = heatmap.fillna(0)  # initialize with zeros

In [ ]:
scores = GCV.cv_results_["mean_test_score"]
for ix, iy, value in zip(x_index, y_index, scores):
    heatmap.loc[iy, ix] += value

In [ ]:
import seaborn as sn
%matplotlib inline

plot = sn.heatmap(heatmap, cmap="hot", annot=True)
plot.set_ylim([len(heatmap), 0])
plot.set_ylabel(p)
plot.set_xlabel(q)

The best value(in this case, the value is some value between 0 and 1) is NOT in the top-left corner. We assum that the parameter values of gamma and degree are good.

In [ ]:
p = params[2]
q = params[3]
y_axis = GCV.param_grid[0][p]
y_index = GCV.cv_results_["param_" + p]

x_axis = GCV.param_grid[0][q]
x_index = GCV.cv_results_["param_" + q]


heatmap = pd.DataFrame(index=y_axis, columns=x_axis, dtype=np.float)
heatmap = heatmap.fillna(0)  # initialize with zeros

scores = GCV.cv_results_["mean_test_score"]
for ix, iy, value in zip(x_index, y_index, scores):
    heatmap.loc[iy, ix] += value
    
import seaborn as sn
%matplotlib inline

plot = sn.heatmap(heatmap, cmap="hot", annot=True)
plot.set_ylim([len(heatmap), 0])
plot.set_ylabel(p)
plot.set_xlabel(q)

The best value(in this case, the value is some value between 0 and 1) is NOT in the top-left corner. We assum that the parameter values of l1_ratio and alpha are good.

Conclusion:
After we using GridSearchCV, we get the best parameters:
{'ENet__alpha': 0.0001,
 'ENet__l1_ratio': 0.1,
 'KPCA__degree': 1,
 'KPCA__gamma': 0.001}
 GCV.score(X,y) = 0.6445953460154428 (0.3215474688612492, the value before searching best parameters)
 GCV.score(X_test,y_test) = 0.6282444866198624 (0.3807477402025766, the value before searching best parameters)

In [ ]:
# use models as parameter

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR

# Create a pipeline
pipe = Pipeline([('classifier', RandomForestRegressor())])

# Create space of candidate learning algorithms and their hyperparameters
param_grid = [{'classifier': [ Ridge()],
               'classifier__alpha':[0.001, 0.01, 0.1, 1,10]
              },
              {'classifier': [RandomForestRegressor()],
               'classifier__n_estimators': [10, 100, 1000],
               'classifier__max_features': [1, 2, 3]},
              {'classifier': [SVR(gamma='scale')],
               'classifier__C':[0.001, 0.01, 0.1, 1, 10]   
              }
             ]

# Create grid search 
clf = GridSearchCV(pipe, param_grid, cv=5, verbose=0)
# Fit grid search
best_model = clf.fit(X, y)
# View best model
print("The best classifier is:\n{}".format(best_model.best_estimator_.get_params()['classifier']))